In [9]:
# code by Tae Hwan Jung @graykode
import torch
import torch.nn as nn
import torch.optim as optim

def make_batch():
    """
    In this very simple V, input = (subject + verb), target = object
    """
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split() # space tokenizer
        input_ = [word_dict[n] for n in word[:-1]] # create (1~n-1) as input
        target = word_dict[word[-1]] # create (n) as target, We usually call this 'casual language model'

        input_batch.append(input_)
        target_batch.append(target)

    return input_batch, target_batch

# Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)                       # mapping C
        self.H = nn.Linear(n_step * m, n_hidden, bias=False)    # hidden layer, eq 1
        self.d = nn.Parameter(torch.ones(n_hidden))             # vector d as in (d + H*x), eq 1, same size as h
        self.U = nn.Linear(n_hidden, n_class, bias=False)       # eq 1, shape = (h, |V|)
        self.W = nn.Linear(n_step * m, n_class, bias=False)     # eq 1
        self.b = nn.Parameter(torch.ones(n_class))              # vector b as in y = b + ..., eq 1, same size as |V|

    def forward(self, X):
        """
        for an input batch X,
        1. create a mapping from words to feature vectors
        2. change shape from  [batch_size, n_step, m] to [batch_size, n_step * m]
        3. calculate eq 1
        output: the unnormalized log-probabilities for each output word i,
        """
        X = self.C(X) # X : [batch_size, n_step, m]
#        print(f"X shape before view(): {X.shape}, {X[0][0][0]},  {X[0][0][1]},  {X[0][1][0]}, {X[0][1][1]}")
        X = X.view(-1, n_step * m) # change the shape from [batch_size, n_step, m] to [batch_size, n_step * m]   
#        print(f"X shape after view(): {X.shape}, {X[0][0]}, {X[0][1]}, {X[0][2]}, {X[0][3]}")
        tanh = torch.tanh(self.d + self.H(X)) # [batch_size, n_hidden]
        output = self.b + self.W(X) + self.U(tanh) # [batch_size, n_class]   # eq 1
        return output

if __name__ == '__main__':
    n_step = 2 # number of steps, n-1 in paper
    n_hidden = 2 # number of hidden units, layers, h in paper
    m = 2 # embedding size, m in paper. Each word is embedded into a vector of size of m

    sentences = ["i like dog", "i love coffee", "i hate milk"] 

    word_list = " ".join(sentences).split()
    word_list = list(set(word_list))                           # deduplicate
    word_dict = {w: i for i, w in enumerate(word_list)}        # so word_dict is our V
    number_dict = {i: w for i, w in enumerate(word_list)}
    n_class = len(word_dict)  # number of Vocabulary, V is the paper, = 7

    model = NNLM()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001) # lr: learning rate

    input_batch, target_batch = make_batch()
    input_batch = torch.LongTensor(input_batch)     # convert to int64
    target_batch = torch.LongTensor(target_batch)

    # Training
    for epoch in range(5000):
        optimizer.zero_grad()          # zero'em
        output = model(input_batch)    # this calls model.forward(input_batch)

        # output : [batch_size, n_class], target_batch : [batch_size]
        loss = criterion(output, target_batch)  # get error
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()                # torch implements back propagation for you
        optimizer.step()
    # at the end of training loop
    
    # Predict
    predict = model(input_batch).data.max(1, keepdim=True)[1]

    # Test
    print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

Epoch: 1000 cost = 0.047264
Epoch: 2000 cost = 0.009019
Epoch: 3000 cost = 0.003305
Epoch: 4000 cost = 0.001523
Epoch: 5000 cost = 0.000780
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
